# Load everything

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import findspark  # Get rid of this in DataBricks
findspark.init()  # Get rid of this in DataBricks
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation


In [2]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.functions import vector_to_array


In [3]:
config = SparkConf().setAll([('spark.executor.memory', '2g'), ('spark.executor.cores', '1'), ('spark.cores.max', '1'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "1");
config.set("spark.dynamicAllocation.maxExecutors", "1");
config.set("spark.dynamicAllocation.initialExecutors", "1"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())  # print all the configuration

[('spark.executor.memory', '2g'), ('spark.dynamicAllocation.minExecutors', '1'), ('spark.driver.port', '43143'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1607107377906'), ('spark.driver.host', '172.18.40.157'), ('spark.cores.max', '1'), ('spark.dynamicAllocation.maxExecutors', '1'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.initialExecutors', '1'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'proj')]


In [7]:
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system

In [8]:
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')

In [9]:
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  # Jjoin them together

# Drop vechile

In [19]:
df = df.filter(df.cp_type == 'trt_cp')
df = df.drop('cp_type')

# OneHot

In [20]:
indexer = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_cat")
df1 = indexer.fit(df).transform(df)
indexer = StringIndexer(inputCol="cp_time", outputCol="cp_time_cat")
df1 = indexer.fit(df1).transform(df1)
df1 = df1.drop('cp_dose')
df1 = df1.drop('cp_time')

encoder = OneHotEncoder(inputCols=["cp_time_cat", "cp_dose_cat"],
                        outputCols=["cp_time_onehot", "cp_dose_onehot"])

model = encoder.fit(df1)
df1 = model.transform(df1)
df1 = df1.withColumn("cp_time_cols", vector_to_array("cp_time_onehot")).select(df1.columns + [col("cp_time_cols")[i] for i in range(2)])
df1 = df1.withColumn("cp_dose_cols", vector_to_array("cp_dose_onehot")).select(df1.columns + [col("cp_dose_cols")[i] for i in range(1)])
df1 = df1.drop('cp_dose_cat',
 'cp_time_cat',
 'cp_time_onehot',
 'cp_dose_onehot',
)


# Feature Engineering  (credit to Jeff)

In [21]:
gene_feature_names = [name for name in df1.columns if 'g-' in name]
cell_feature_names =  [name for name in df1.columns if 'c-' in name]

df2 = df1.withColumn("gene_max", F.greatest(*gene_feature_names))
df2 = df2.withColumn("gene_min", F.least(*gene_feature_names))
df2 = df2.withColumn("cell_max", F.greatest(*cell_feature_names))
df2 = df2.withColumn("cell_min", F.least(*cell_feature_names))


# Drop high correlation features

In [22]:
feature_columns = gene_feature_names + cell_feature_names  # This came from the previous section
vectorAssembler = VectorAssembler(inputCols = feature_columns, outputCol = 'feats' )
feature_vector = vectorAssembler.transform(df2).select("feats")
# pyspark implementation of determining the correlations
corr_matrix = Correlation.corr(feature_vector, "feats").head()[0]

# Convert the correlation desne matrix and apply mask and to get the indicies where high correlations are observed
# In here, I convert the correlation matrix to numpy, and then use numpy's mask to obtain the lower traingle of the
# matrix. I used numpy becasue pyspark does not have mask.

corr_Array = corr_matrix.toArray()
masked_corr = np.ma.masked_where(np.triu(np.ones_like(corr_Array, dtype=bool)), corr_Array, copy=True)  
idx_high_corr_feats = set(np.argwhere(masked_corr > 0.90)[:,0])  # Set threshold to 90%

# Identify the column to drop and then drop it.
features_to_drop = np.array(feature_columns)[list(idx_high_corr_feats)].tolist()


# Finally

df3 = df2.drop(*features_to_drop)

# Feature trans

???

# Training

In [28]:
final_feature_names = list(set(df3.columns) - set(df_label.columns))

In [29]:
vectorAssembler = VectorAssembler(inputCols = final_feature_names, outputCol = 'feats' )
df4 = vectorAssembler.transform(df3).drop(*final_feature_names)

In [32]:
individual_label_name =  '5-alpha_reductase_inhibitor'


In [33]:
temp_df = df4.select('feats', individual_label_name)

In [ ]:
# Stratify 
# ref: https://stackoverflow.com/questions/47637760/stratified-sampling-with-pyspark/47672336

In [58]:
seed = 42
fractions = {1: 0.8, 0: 0.8}
train_df = temp_df.stat.sampleBy(individual_label_name, fractions, seed, ).cache()
test_df =  temp_df.subtract(train_df).cache()

In [60]:
train_df.sample(True, 1.1, 42)


DataFrame[feats: vector, 5-alpha_reductase_inhibitor: int]

In [44]:
df4.count()

21948

In [51]:
test_df.count()

4265

# SCRAP

In [60]:
my_set = {'Geeks', 'for', 'geeks'} 
  
s = list(my_set)

TypeError: 'list' object is not callable

In [62]:
type(list)

list

In [ ]:
cell_feature_names

In [32]:
df.select("cp_type").distinct().show()

+-------+
|cp_type|
+-------+
| trt_cp|
+-------+



In [ ]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [6]:
from pyspark.sql.functions import lit


In [7]:
lit(0.8)

Column<b'0.8'>

In [8]:
from pyspark.sql.functions import lit
list = [(2147481832,23355149,1),(2147481832,973010692,1),(2147481832,2134870842,1),(2147481832,541023347,1),(2147481832,1682206630,1),(2147481832,1138211459,1),(2147481832,852202566,1),(2147481832,201375938,1),(2147481832,486538879,1),(2147481832,919187908,1),(214748183,919187908,1),(214748183,91187908,1)]
df = spark.createDataFrame(list, ["x1","x2","x3"])

In [9]:
df

DataFrame[x1: bigint, x2: bigint, x3: bigint]

In [10]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [12]:
fractions

{2147481832: 0.8, 214748183: 0.8}

In [15]:
df.select("x1").distinct().withColumn("fraction", lit(0.8)).show()

+----------+--------+
|        x1|fraction|
+----------+--------+
|2147481832|     0.8|
| 214748183|     0.8|
+----------+--------+



In [ ]:
temp_df = features_and_targets.withColumn('target_vector', (vector_to_string(array([features_and_targets[col] for col in target_names])))).select(['sig_id', 'target_vector'])
string_indexer = StringIndexer(inputCol = 'target_vector', outputCol = 'target')
string_indexer_model = string_indexer.fit(temp_df)
temp_df = string_indexer_model.transform(temp_df).drop('target_vector')

data = features_and_targets.join(temp_df, features_and_targets.sig_id == temp_df.sig_id, how = 'inner').drop(temp_df.sig_id)